## Window Function

In [38]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window


### Criar/Iniciar sessão do pyspark

In [39]:
spark = (
    SparkSession.builder
    .master('local')
    .appName('aula02')
    .getOrCreate()
)

In [40]:
df = spark.read.csv('../datasets/equipamentos.csv', header=True, inferSchema=True, sep=',')

In [41]:
df.show(5)

+--------------+------+---+-------+--------------------+-----------+--------+--------------------+------+---------------+--------------------+-------------+--------+--------------------+--------------------+
|id_equipamento|  ibge| uf| cidade|                nome|responsavel|telefone|            endereco|numero|    complemento|          referencia|       bairro|     cep|     georef_location|    data_atualizacao|
+--------------+------+---+-------+--------------------+-----------+--------+--------------------+------+---------------+--------------------+-------------+--------+--------------------+--------------------+
|   23001006462|230010| CE|Abaiara|       CRAS I - SEDE|       NULL|    NULL|JOAQUIM LEITE DA ...|   268|           NULL|PRÓXIMO A SECRETA...|       CENTRO|63240000|-7.36175927968484...|2024-05-09 01:00:...|
|   23001020547|230010| CE|Abaiara|             CRAS II|       NULL|    NULL|CE, 393, SITIO BR...|     0|           NULL|PRÓXIMO A CAPELA ...|Vila São José|63240000|-7.

In [42]:
df.printSchema()

root
 |-- id_equipamento: long (nullable = true)
 |-- ibge: integer (nullable = true)
 |-- uf: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- nome: string (nullable = true)
 |-- responsavel: string (nullable = true)
 |-- telefone: string (nullable = true)
 |-- endereco: string (nullable = true)
 |-- numero: integer (nullable = true)
 |-- complemento: string (nullable = true)
 |-- referencia: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: integer (nullable = true)
 |-- georef_location: string (nullable = true)
 |-- data_atualizacao: timestamp (nullable = true)



In [43]:
for column in df.columns:
    print(column, df.filter(df[column].isNull()).count())

id_equipamento 0
ibge 0
uf 0
cidade 0
nome 0
responsavel 406
telefone 406
endereco 0
numero 0
complemento 315
referencia 135
bairro 0
cep 0
georef_location 13
data_atualizacao 0


In [48]:
df.filter(col('georef_location').isNull()).show()

+--------------+------+---+---------------+--------------------+-----------+--------+--------------------+------+-----------+--------------------+--------------------+--------+---------------+--------------------+
|id_equipamento|  ibge| uf|         cidade|                nome|responsavel|telefone|            endereco|numero|complemento|          referencia|              bairro|     cep|georef_location|    data_atualizacao|
+--------------+------+---+---------------+--------------------+-----------+--------+--------------------+------+-----------+--------------------+--------------------+--------+---------------+--------------------+
|   23021039721|230210| CE|       Baturité|CRAS - CONSELHEIR...|       NULL|    NULL|FRANCISCO BRAGA F...|     0|       NULL|VIZINHO A SECRETA...|CONSELHEIRO ESTELITA|62760000|           NULL|2024-05-09 01:00:...|
|   23027039321|230270| CE|   Campos Sales|CRAS - Miguel Cortez|       NULL|    NULL|Conjunto Lindalva...|     0|       NULL|                NUL

In [60]:
complemento = lambda x : 'sem complemento' if x is None else x
referencia  = lambda x : 'sem referencia' if x is None else x
georef_location = lambda x : 'indefinido' if x is None else x

fill_complemento = complemento

fill_referencia = referencia
fill_georef_location = georef_location


In [50]:
df.show()

+--------------+------+---+-----------------+--------------------+-----------+--------+--------------------+------+---------------+--------------------+--------------------+--------+--------------------+--------------------+
|id_equipamento|  ibge| uf|           cidade|                nome|responsavel|telefone|            endereco|numero|    complemento|          referencia|              bairro|     cep|     georef_location|    data_atualizacao|
+--------------+------+---+-----------------+--------------------+-----------+--------+--------------------+------+---------------+--------------------+--------------------+--------+--------------------+--------------------+
|   23001006462|230010| CE|          Abaiara|       CRAS I - SEDE|       NULL|    NULL|JOAQUIM LEITE DA ...|   268|           NULL|PRÓXIMO A SECRETA...|              CENTRO|63240000|-7.36175927968484...|2024-05-09 01:00:...|
|   23001020547|230010| CE|          Abaiara|             CRAS II|       NULL|    NULL|CE, 393, SITI

In [61]:
filled_df = df.select(col())

AttributeError: 'function' object has no attribute '_get_object_id'

In [ ]:
df.show()